In [ ]:
### input: folder with files, that contain events with unique anonymousId
### output: csv with columns: time_interval (in sec), ip (boolean), userId (boolean), fingerPrint (boolean) 

### P.S. boolean -> 1 if chaged, 0 if not

In [3]:
import pandas as pd
import csv
import json
import glob
import os

In [70]:
files = [os.path.basename(x) for x in glob.glob('./users/*.json')]
print "ok"

result_file = "time_intervals.csv"
with open(result_file, "wb") as f:
    writer = csv.DictWriter(f, fieldnames = ["interval (in sec)", "ip", "userId", "fingerPrint"])
    writer.writeheader()

for file_name in files:
    f = open('./users/%s'%file_name)
    context = f.readlines()
    if (len(context) -1) > 1:
        data = []
        for i in context[1:]:
            data += [json.loads(i.replace(',\n','').replace('\n',''))]
        
        data = pd.DataFrame.from_records(data)

        if 'fingerPrint' not in data.keys(): data['fingerPrint'] = [0 for i in range(data.shape[0])]
        data = pd.concat([data['timestamp'], data['ip'], data['userId'], data['fingerPrint']], axis=1)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        
        sorted_data = data.sort(['timestamp']).values.tolist()
        
        new_data = []
        for i in range(1, len(sorted_data)):
            new_data.append([(sorted_data[i][0] - sorted_data[i-1][0]).seconds, 0 if sorted_data[i][1] == sorted_data[i-1][1] else 1,
                            0 if sorted_data[i][2] == sorted_data[i-1][2] else 1,
                            0 if sorted_data[i][3] == sorted_data[i-1][3] else 1])
        
        with open(result_file, "a") as f:
            writer = csv.writer(f)
            writer.writerows(new_data)

print "Ready!"

ok
Ready!
